## Model Data using Cassandra


### Please just submit this notebook in the Submission and make sure each cell has been executed and output is clearly displayed.

### The aim of the project is to solve the three queries given below.

### Introduction

There is a music streaming app called SoundCloud, that has been using their music streaming app and collecting data on songs and user activity and their aim is to analyze this data especially understanding what songs users are listening to. Currently, they are not making use of a NoSQL db and they have the data stored as a CSV file, thus its difficult for them to query the data. So our task is to create a NoSQL database for helping them with the analysis.

#### Import Packages 

In [1]:
import pandas as pd
import numpy as np
import cassandra
import csv


##  The image below is a screenshot of what the data appears like in the event_data.csv

<img src="event_data_image.jpg">

#### Creating a Cluster

In [2]:
# Task: Make a connection to the cassandra instance on your local machine(127.0.0.1) and 
# create a session to establish connection and begin executing queries

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cluster = Cluster(contact_points=['127.0.0.1'], port=9042)
session = cluster.connect()

#### Create & Set Keyspace

In [3]:
# Task: Create a Keyspace and Set KEYSPACE to the keyspace specified above
session.execute("DROP KEYSPACE IF EXISTS test")
session.execute("CREATE KEYSPACE test WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 1};")

## List of Queries 

### 1. Find the artist_name, song_title and length of song the SoundCloud app history that was heard during  session_number = 338, and item_in_session_number  = 4


### 2. Find the artist_name, song_title (sorted by item_in_session_number) and name(fname and lname) of the user for user_id = 10, session_number = 182
    

### 3. Find every name(fname and lname) of the user from the SoundCloud app history that listened to the song_title 'All Hands Against His Own'




### Query1 Table1: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [4]:
## Task: Query 1: Find the artist_name, song_title and length of song the SoundCloud app history 
## that was heard during session_number = 338, and item_in_session_number = 4
## make use of create table command
q = """
CREATE TABLE IF NOT EXISTS test.query1_table1(
    artist_name text
    ,item_in_session_number int
    ,length float
    ,session_number int
    ,song_title text
    ,PRIMARY KEY (session_number, item_in_session_number)
    );
"""

try:
    session.execute(q)
except Exception as e:
    print(e)

rows = session.execute("select count(*) from test.query1_table1;")
for row in rows:
    print(row)

Row(count=0)


### Let's insert our data into of table

In [5]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = '/Users/francis/Documents/event_data.csv'

cnt=0
with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
        ## Task: Write the INSERT statements and assign it to the query variable
        q = """
        INSERT INTO test.query1_table1 (session_number, item_in_session_number, artist_name, song_title, length) 
        VALUES (%s, %s, %s, %s, %s)
        """
#         session_number 8
#         item_in_session_number 3
#         artist_name 0 
#         song_title 9 
#         length 5
        cnt += 1
        try:
            session.execute(q, (int(row[8]), int(row[3]), row[0], row[9], float(row[5])))
        except Exception as e:
            print(e)
    
print("[%d] rows processed" % cnt)
rows = session.execute("select count(*) from test.query1_table1;")
for row in rows:
    print(row)

[6820] rows processed
Row(count=6820)


### Validate our Data Model using a SELECT

In [6]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

try:
    rows = session.execute("""
    select 
        artist_name
        ,song_title
        ,length 
    from test.query1_table1 
    where session_number=338 
        and item_in_session_number = 4
    ;
    """)
    for row in rows:
        print("%-20s %-45s %.2f" % (row.artist_name, row.song_title, row.length,))
except Exception as e:
    print(e)

Faithless            Music Matters (Mark Knight Dub)               495.31


### Query2 Table2: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key 

In [7]:
## Task: Query 2: Find the artist_name, song_title (sorted by item_in_session_number) and 
## name(fname and lname) of the user for user_id = 10, session_number = 182 
## make use of create table command
q = """
CREATE TABLE IF NOT EXISTS test.query2_table2(
    artist_name text
    ,item_in_session_number int
    ,length float
    ,session_number int
    ,song_title text
    ,fname text
    ,lname text
    ,user_id int
    ,PRIMARY KEY (user_id, session_number, item_in_session_number)
    )
;
"""

try:
    session.execute(q)
except Exception as e:
    print(e)
    
rows = session.execute("select count(*) from test.query2_table2;")
for row in rows:
    print(row)

Row(count=0)


### Let's insert our data into of table

In [8]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

cnt=0
with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
        q = """
        INSERT INTO test.query2_table2 (
        user_id, session_number, item_in_session_number, artist_name, length, song_title, fname, lname) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        # user_id, 10 
        # session_number, 8
        # item_in_session_number, 3
        # artist_name, 0
        # length, 5
        # song_title, 9
        # fname, 1
        # lname, 4
        cnt+=1
        try:
            session.execute(q, (int(row[10]), int(row[8]), int(row[3]), row[0]
                                , float(row[5]), row[9], row[1], row[4]))
        except Exception as e:
            print(e)
print("[%d] rows processed" % cnt)
rows = session.execute("select count(*) from test.query2_table2;")
for row in rows:
    print(row)

[6820] rows processed
Row(count=6820)


### Validate our Data Model using a SELECT

In [9]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

try:
    rows = session.execute("""
    select 
        artist_name
        ,song_title
        ,item_in_session_number
        ,fname
        ,lname 
    from test.query2_table2 
    where user_id=10 
        and session_number = 182
    ;
    """)
    for row in rows:
        print("%-35s %-55s %5s %s %s" % (row.artist_name, 
                                         row.song_title, 
                                         row.item_in_session_number, 
                                         row.fname, 
                                         row.lname,))
except Exception as e:
    print(e)

Down To The Bone                    Keep On Keepin' On                                          0 Sylvie Cruz
Three Drives                        Greece 2000                                                 1 Sylvie Cruz
Sebastien Tellier                   Kilometer                                                   2 Sylvie Cruz
Lonnie Gordon                       Catch You Baby (Steve Pitron & Max Sanna Radio Edit)        3 Sylvie Cruz


### Query3 Table3: How should we model this data? Think about what should be our Primary Key/Partition Key/Clustering Key

In [10]:
## Task: Query 3: Find every name(first and lastname) of the user from the SoundCloud app history that listened 
## to the song_title 'All Hands Against His Own'
## make use of create table command    
q = """
CREATE TABLE IF NOT EXISTS test.query3_table3(
    song_title text
    ,session_number int
    ,item_in_session_number int
    ,fname text
    ,lname text
    ,PRIMARY KEY (song_title, session_number, item_in_session_number)
    )
;
"""

try:
    session.execute(q)
except Exception as e:
    print(e)
    
rows = session.execute("select count(*) from test.query3_table3;")
for row in rows:
    print(row)

Row(count=0)


### Let's insert our data into of table

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file_name = 'event_data.csv'

cnt=0
with open(file_name, encoding = 'utf8') as f:
    csv_reader = csv.reader(f)
    next(csv_reader) # skip the header in the csv file
    for row in csv_reader:
        q = """
        INSERT INTO test.query3_table3 (
            song_title
            ,session_number
            ,item_in_session_number
            ,fname
            ,lname
        ) 
        VALUES (%s, %s, %s, %s, %s)
        """
#         song_title 9
#         ,session_number 8
#         ,item_in_session_number 3
#         ,fname 1
#         ,lname 4
        cnt+=1
        try:
            session.execute(q, (row[9], int(row[8]), int(row[3]), row[1], row[4]))
        except Exception as e:
            print(e)
print("[%d] rows processed" % cnt)
rows = session.execute("select count(*) from test.query3_table3;")
for row in rows:
    print(row)

[6820] rows processed
Row(count=6820)


### Validate our Data Model using a SELECT

In [12]:
## Task: Make use of the SELECT statement and for loop to check if your query works and display the results

try:
    rows = session.execute("""
    select 
        song_title
        ,fname
        ,lname 
        ,session_number
        ,item_in_session_number
    from test.query3_table3 
    where song_title = 'All Hands Against His Own'
    ;
    """)
    for row in rows:
        print("%-30s %-15s %-10s" % (row.song_title, row.fname, row.lname))
except Exception as e:
    print(e)
    

All Hands Against His Own      Sara            Johnson   
All Hands Against His Own      Jacqueline      Lynch     
All Hands Against His Own      Tegan           Levine    


### Drop the tables before closing out the sessions

In [13]:
rows = list(session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name = 'test'"))
print("%d tables in keyspace test" % len(rows))
for row in rows:
    print("%s.%s" % (row.keyspace_name,row.table_name))

3 tables in keyspace test
test.query1_table1
test.query2_table2
test.query3_table3


In [14]:
for row in rows:
    q = "drop table %s.%s;" % (row.keyspace_name,row.table_name)
    print("%s ... ... " % q, end="")
    session.execute(q)
    print("Done.", end="\n")

drop table test.query1_table1; ... ... Done.
drop table test.query2_table2; ... ... Done.
drop table test.query3_table3; ... ... Done.


In [15]:
rows = list(session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name = 'test'"))
print("%d tables in keyspace test" % len(rows))
for row in rows:
    print("%s.%s" % (row.keyspace_name,row.table_name))

0 tables in keyspace test


### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()